# tensorflow layers from birdnet

See [this notebook from Kien](https://github.com/dsgt-birdclef/birdclef-eda-f22/blob/bd88ada017a7018351ca06ee675bc34d1743f620/users/kientran/notebooks/BirdnetExplore.ipynb).

In [ ]:
import os
from pathlib import Path

import requests
import numpy as np
import tensorflow as tf
from tensorflow import keras


os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [ ]:
checkpoint_path = Path("../data/raw/birdnet/v2.2/checkpoint")

checkpoint_base_url = "https://github.com/kahst/BirdNET-Analyzer/raw/main/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model"
for part in [
    "saved_model.pb",
    "variables/variables.index",
    "variables/variables.data-00000-of-00001",
]:
    # get response content and save to disk in checkpoint path
    content = requests.get(f"{checkpoint_base_url}/{part}").content
    p = checkpoint_path / part
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_bytes(content)

print([p.as_posix() for p in checkpoint_path.glob("**/*")])

In [1]:
checkpoint_path = Path("../data/raw/birdnet/v2.2/checkpoint")
model = tf.saved_model.load(checkpoint_path, tags=["serve"])

In [2]:
# [x for x in dir(model) if not x.startswith("_")]
# [x for x in dir(model.model) if not x.startswith("_")]
[v.name for v in model.model.variables]

['ADVANCED_SPEC1/magnitude_scaling:0',
 'BNORM_SPEC_NOQUANT/gamma:0',
 'BNORM_SPEC_NOQUANT/beta:0',
 'BNORM_SPEC_NOQUANT/moving_mean:0',
 'BNORM_SPEC_NOQUANT/moving_variance:0',
 'CONV_0/kernel:0',
 'BNORM_0/gamma:0',
 'BNORM_0/beta:0',
 'BNORM_0/moving_mean:0',
 'BNORM_0/moving_variance:0',
 'pool_0_CONV/kernel:0',
 'pool_0_CONV/bias:0',
 'BLOCK_1-1_CONV_1/kernel:0',
 'BLOCK_1-1_BN_1/gamma:0',
 'BLOCK_1-1_BN_1/beta:0',
 'BLOCK_1-1_BN_1/moving_mean:0',
 'BLOCK_1-1_BN_1/moving_variance:0',
 'BLOCK_1-1_CONV_3/kernel:0',
 'BLOCK_1-1_BN_3/gamma:0',
 'BLOCK_1-1_BN_3/beta:0',
 'BLOCK_1-1_BN_3/moving_mean:0',
 'BLOCK_1-1_BN_3/moving_variance:0',
 'BLOCK_1-2_CONV_1/kernel:0',
 'BLOCK_1-2_BN_1/gamma:0',
 'BLOCK_1-2_BN_1/beta:0',
 'BLOCK_1-2_BN_1/moving_mean:0',
 'BLOCK_1-2_BN_1/moving_variance:0',
 'BLOCK_1-2_CONV_3/kernel:0',
 'BLOCK_1-2_BN_3/gamma:0',
 'BLOCK_1-2_BN_3/beta:0',
 'BLOCK_1-2_BN_3/moving_mean:0',
 'BLOCK_1-2_BN_3/moving_variance:0',
 'BLOCK_1-3_CONV_1/kernel:0',
 'BLOCK_1-3_BN_1/

## using keras

Lets see how birdnet does it. We see that they use the [keras interface to load from the protobuf file](https://github.com/kahst/BirdNET-Analyzer/blob/main/model.py).

In [3]:
pb_model = keras.models.load_model(checkpoint_path)
pb_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x20e19ad68c0>

In [4]:
pb_model.model.layers[:10]

In [5]:
pb_model.model.layers[0].get_config()

{'batch_input_shape': (None, 144000),
 'dtype': 'float32',
 'sparse': False,
 'ragged': False,
 'name': 'INPUT'}

Interesting error popped up when I tried this.

```bash
InternalError: Graph execution error:
...
    File "f:\code\kaggle\birdclef-eda-f22\users\acmiyaguchi\venv\lib\site-packages\keras\layers\normalization\batch_normalization.py", line 649, in _fused_batch_norm_inference
      return tf.compat.v1.nn.fused_batch_norm(
Node: 'model/BNORM_SPEC_NOQUANT/FusedBatchNormV3'
cuDNN launch failure : input shape ([1,1,128,513])
	 [[{{node model/BNORM_SPEC_NOQUANT/FusedBatchNormV3}}]] [Op:__inference_predict_function_56499]
```

- https://github.com/DeepLabCut/DeepLabCut/issues/1

In [6]:
input_shape = pb_model.model.layers[0].get_config()["batch_input_shape"][1]
sample = np.random.rand(1, input_shape)

pb_model.model.predict(sample)

InternalError: Graph execution error:

Detected at node 'model/BNORM_SPEC_NOQUANT/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\acmiy\.pyenv\pyenv-win\versions\3.10.5\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\acmiy\.pyenv\pyenv-win\versions\3.10.5\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "f:\code\kaggle\birdclef-eda-f22\users\acmiyaguchi\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "f:\code\kaggle\birdclef-eda-f22\users\acmiyaguchi\venv\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "f:\code\kaggle\birdclef-eda-f22\users\acmiyaguchi\venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "f:\code\kaggle\birdclef-eda-f22\users\acmiyaguchi\venv\lib\site-packages\tornado\platform\asynci